In [11]:
import re
y = ["Ala542_Lysext-5", "Ala542_Lys543", "Ala542_Lysfs", "Ala542_Lys543insAlaLysAlaAla"]
print(y)
d = {"Ala": "A", "Lys": "K"}
def replace_by_dict (mut, dict_aa):
    for k,v in dict_aa.items():
        mut = mut.replace(k,v)
    return mut
y2 = [replace_by_dict(mut, d) for mut in y]
print(y2)


# Create a regular expression from all of the dictionary keys
regex = re.compile("|".join(map(re.escape, d.keys())))
# For each match, look up the corresponding value in the dictionary
y3 = [regex.sub(lambda match: d[match.group(0)], mut) for mut in y]
print(y3)

['Ala542_Lysext-5', 'Ala542_Lys543', 'Ala542_Lysfs', 'Ala542_Lys543insAlaLysAlaAla']
['A542_Kext-5', 'A542_K543', 'A542_Kfs', 'A542_K543insAKAA']
['A542_Kext-5', 'A542_K543', 'A542_Kfs', 'A542_K543insAKAA']


In [1]:
import pandas as pd
import numpy as np
import re

p_prot = pd.read_csv("../db_tables/protein.tsv", sep="\t")
p_prot = p_prot[['id_protein', 'uniprot_acc']]

#by ensemble transcript
protein_enst = pd.read_csv("../raw_data/llps_uniprot2ENST.tab.txt", sep="\t")
protein_enst = protein_enst.rename(columns={'From': 'uniprot_acc', 'To':'ensembl_enst'})
protein_enst = protein_enst.merge(p_prot[['id_protein', 'uniprot_acc']]).drop(columns=['uniprot_acc'])

#by ensemble protein
protein_ensp = pd.read_csv("../raw_data/llps_uniprot2ENSP.tab.txt", sep="\t")
protein_ensp = protein_ensp.rename(columns={'From': 'uniprot_acc', 'To':'ensembl_ensp'})
protein_ensp = protein_ensp.merge(p_prot[['id_protein', 'uniprot_acc']]).drop(columns=['uniprot_acc'])





datafile  = "../raw_data/cosmic_v94/CosmicMutantExport_sel.tsv"
#for each sample_id has the cancer types columns and the cosmic_phenotype_id
samples_data = "../raw_data/cosmic_v94/CosmicSample.tsv" 
#for each cosmic_phenotype_id has the NCI_CODE and other like EFO, MONDO, ....
class_data = "../raw_data/cosmic_v94/classification.csv"

In [2]:
mutation = pd.read_csv(datafile, sep="\t")
#to string the columns
mutation['pubmed'] = mutation['pubmed'].map(lambda x: str(int(x)) if not np.isnan(x) else x)
print(list(mutation.columns))
print(mutation.head())
print(f'cosmic rows {mutation.shape[0]}') 

['id_sample', 'GENOMIC_MUTATION_ID', 'LEGACY_MUTATION_ID', 'MUTATION_ID', 'CDS', 'AA', 'consequence', 'genomic', 'somatic_status', 'pubmed', 'ensembl_ensp', 'ensembl_enst']
   id_sample GENOMIC_MUTATION_ID LEGACY_MUTATION_ID  MUTATION_ID  \
0    2198328        COSV57322711        COSM1628357     98293492   
1    2869867        COSV57321510        COSM1706236     98292376   
2    2549254        COSV57321745        COSM5796874     98292486   
3    2749203        COSV57324149        COSM7003726     98292523   
4    2839416       COSV104562760        COSM9463514     98293844   

                 CDS       AA              consequence  \
0           c.609G>T  p.M203I  Substitution - Missense   
1           c.781G>A  p.E261K  Substitution - Missense   
2           c.385G>C  p.G129R  Substitution - Missense   
3            c.20G>C    p.G7A  Substitution - Missense   
4  c.559_560delinsAA  p.G187K  Substitution - Missense   

                  genomic             somatic_status    pubmed  \
0  

In [3]:
# Filter only mutations in our LLPS dataset 

mutation['internalid'] = range(1, len(mutation)+1)

#merge by order 'ensembl_enst', 'ensembl_ensp'
#to avoid the rows duplication
protein_enst = mutation.merge(protein_enst)
mapped_internal = protein_enst['internalid'].unique().tolist()
mutation = mutation[~mutation['internalid'].isin(mapped_internal)]
print(f'rows mapped by ENST_name {protein_enst.shape[0]}')

#analizar enst con disitino id_protein
y =  protein_enst[['ensembl_enst', 'id_protein']]
y = y.drop_duplicates()
dd = y.groupby(['ensembl_enst']).size().reset_index(name='counts')
print(dd[dd.counts > 1])
#este es un caso raro ya que la secuencia del transcripto es la union de dos uniprots
#P35544 + P62861
#id_protein 4340 y 4342

protein_ensp = mutation.merge(protein_ensp)
mapped_internal = protein_ensp['internalid'].unique().tolist()
mutation = mutation[~mutation['internalid'].isin(mapped_internal)]
print(f'rows mapped by ENSP_name {protein_ensp.shape[0]}')
#analisar ensp con disitino id_protein
y =  protein_ensp[['ensembl_ensp', 'id_protein']]
y = y.drop_duplicates()
dd = y.groupby(['ensembl_ensp']).size().reset_index(name='counts')
print(dd[dd.counts > 1])

#mapear por gene_name, gene_id, hgnc_id


mutation = pd.concat([protein_enst, protein_ensp], ignore_index=True)
mutation = mutation.drop_duplicates()
print(f'cosmic without duplicates {mutation.shape[0]}') 
print(f'cosmic proteins {len(mutation["id_protein"].unique())}')                


rows mapped by ENST_name 1724388
         ensembl_enst  counts
3692  ENST00000529639       2
rows mapped by ENSP_name 0
Empty DataFrame
Columns: [ensembl_ensp, counts]
Index: []
cosmic without duplicates 1724388
cosmic proteins 4018


In [4]:
print(protein_enst[protein_enst['ensembl_enst'] == 'ENST00000529639']['AA'].unique())

['p.V76=' 'p.K102del' 'p.P91S' 'p.E37*' 'p.G78C' 'p.P36S' 'p.P129L'
 'p.G128D' 'p.Q96=' 'p.T53I' 'p.A61V' 'p.K127M' 'p.V21G' 'p.Y112C'
 'p.R87T' 'p.A26V' 'p.K25N' 'p.E37D' 'p.R82H' 'p.E97=' 'p.P129H' 'p.Y112H'
 'p.E9V' 'p.K102T' 'p.A26G' 'p.L3=' 'p.R109Q' 'p.A131T' 'p.R109W' 'p.P36L'
 'p.S133C' 'p.A29=' 'p.P36T' 'p.R108W' 'p.R6P' 'p.R6=' 'p.R114W' 'p.A81='
 'p.H77R' 'p.K99N' 'p.H27D' 'p.Q111*' 'p.K101=' 'p.K98=' 'p.V86='
 'p.K100=' 'p.A61T' 'p.N118=' 'p.G58E' 'p.N132Y' 'p.K92E' 'p.R115H'
 'p.E66K']


In [5]:
#ver si hay mutiples transcriptos de una misma proteina para eliminarlos 
#un legacy_mutation_id tiene todas los transcriptos mutados que corresponde a una misma posicion
y = mutation[['LEGACY_MUTATION_ID', 'id_protein']]
y = y.drop_duplicates()
print(y.shape[0])
mut_trans = y.groupby(['LEGACY_MUTATION_ID']).size().reset_index(name='counts')
mut_trans = mut_trans[mut_trans.counts > 1]
print(mut_trans)
#puede darse el caso que tengan varios protein_id ya que los uniprot asociados son disitntas variantes de splicing
y = mutation[['LEGACY_MUTATION_ID', 'id_protein', 'ensembl_enst']]
y = y.drop_duplicates()
mut_trans = y.groupby(['LEGACY_MUTATION_ID', 'id_protein']).size().reset_index(name='counts')
mut_trans = mut_trans[mut_trans.counts > 1]
print(mut_trans)
#only one transcript per protein id


1144756
        LEGACY_MUTATION_ID  counts
8381           COSM1049194       2
8382           COSM1049196       2
33704          COSM1206683       2
59817          COSM1355890       2
85748           COSM160375       2
...                    ...     ...
1063915        COSM9462306       2
1071035        COSM9508187       2
1084998        COSM9589314       2
1090340        COSM9621560       2
1099092         COSM967422       2

[81 rows x 2 columns]
Empty DataFrame
Columns: [LEGACY_MUTATION_ID, id_protein, counts]
Index: []


In [6]:
#ver si id_mutation and id_protein es unique de la tabla
#de ser asi, dejar solo id_mutation y datos de las mutaciones juntas
#y otra tabla con "ID_sample", "GENOMIC_MUTATION_ID", "LEGACY_MUTATION_ID", "MUTATION_ID", "Mutation somatic status", "Pubmed_PMID"

samples_mut = mutation[["id_sample", "LEGACY_MUTATION_ID", 'id_protein', "somatic_status", "pubmed"]]
samples_mut = samples_mut.drop_duplicates()
print(f'sample-mutation {samples_mut.shape[0]}')
#samples_mut.to_csv('samples_mut.tsv', sep="\t", index= False)
#"LEGACY_MUTATION_ID" + 'id_protein' is the mutation id
mutation = mutation[["LEGACY_MUTATION_ID", 'id_protein', 'CDS', 'AA', 'consequence', 'genomic']]
mutation = mutation.drop_duplicates()
print(f'mutation-idprot-transcr {mutation.shape[0]}')


sample-mutation 1723613
mutation-idprot-transcr 1144756


In [7]:
#split genomic by (x:y-z) in chromosome, start_genomic end_genomic
mutation['genomic'] = mutation['genomic'].str.findall('^(.+):(.+)-(.+)$').str[0]
mutation['chromosome'] = mutation['genomic'].map(lambda x: x[0])
mutation['start_genomic'] = mutation['genomic'].map(lambda x: x[1])
mutation['end_genomic'] = mutation['genomic'].map(lambda x: x[2])
mutation = mutation.drop(columns=['genomic'])
print(mutation['chromosome'].unique().tolist())

['10', '2', '15', '11', '19', '7', '17', '1', '3', '12', '14', '5', '9', '4', '16', '6', '22', '21', '18', '8', '20', '23', '13', '24', '25']


In [8]:
print(mutation.shape[0])
mutation = mutation.rename(columns={'consequence': 'consequence_cosmic'})
mutation['indexinternal'] = range(1, len(mutation)+1)
mutation['AA'] = mutation.AA.map(lambda x: re.findall('^p\.(.*)$', x))
mutation['AA'] = mutation.AA.str[0]
mutation = mutation[mutation.AA.notnull()]

mutation['CDS'] = mutation.CDS.map(lambda x: re.findall('^c\.(.*)$', x))
mutation['CDS'] = mutation.CDS.str[0]
mutation = mutation[mutation.CDS.notnull()]

print(mutation.shape[0])
print(mutation)



1144756
1144756
        LEGACY_MUTATION_ID  id_protein  \
0               COSM166059        1011   
2              COSM6982134        1011   
3               COSM428012        1011   
5               COSM427996        1011   
6              COSM5703736        1011   
...                    ...         ...   
1724382        COSM7772448        1844   
1724383        COSM4622984        1844   
1724384        COSM9330544        1844   
1724386        COSM8832530        1844   
1724387        COSM3944131        1844   

                                                       CDS  \
0                                                  1224dup   
2        661_662delinsCCTGCCTGTTGTGAGCTGCTCTACGTGCCCTAC...   
3                                             1302_1303dup   
5                                                  1203dup   
6                                                   633del   
...                                                    ...   
1724382                                      

In [9]:
def separar_en_cols(df, column, conseq, conseq_regex, override=False):
    '''
    recibe un DataFrame, el nombre de una columna auxiliar (column)
    y un string con el tipo de consecuencia (conseq). La col. auxiliar
    es una tupla con los elementos implicados en una mutacion
    como la siguiente (aa1, start_pos, aa2, end_pos, aa/s_nuevos).
    Devuelve el DataFrame df con estas 5 nuevas columnas
    '''
    
    df_crop = df.copy()
      
      
    if override:
        df_crop['aux'] = df_crop[column].str.findall(conseq_regex).str[0]
    else:
        df_crop['aux'] = df_crop[column].str.findall('^([A-Z\*])(\d+)_?([A-Z\*])?(\d+)?'+conseq_regex+'(.*)$').str[0]
  
    df_crop = df_crop[~df_crop['aux'].isnull()]
    '''
    #check list type bad format    
    if conseq in ['synonym', 'nonsense']: #(w, d, '', '', '')
        df_crop['badformat'] = df_crop['aux'].map(lambda x: True if ((x[0] == '') or (x[1] == '') or (x[2] != '') or (x[3] != '') or (x[4] != '')) else False)
    elif conseq in ['delins']: #(w, d, '', '', w) or (w, d, w, d, w)
        df_crop['badformat'] = df_crop['aux'].map(lambda x: True if (((x[0] == '') or (x[1] == '') or (x[4] == '')) or ((x[2] != '') and (x[3] == '')) or ((x[2] == '') and (x[3] != ''))) else False) 
    elif conseq in ['deletion', 'duplication']: #(w, d, '', '', '') or (w, d, w, d, '')
        df_crop['badformat'] = df_crop['aux'].map(lambda x: True if (((x[0] == '') or (x[1] == '') or (x[4] != '')) or ((x[2] != '') and (x[3] == '')) or ((x[2] == '') and (x[3] != ''))) else False) 
    elif conseq in ['insertion']: #(w, d, w, d, w)
        df_crop['badformat'] = df_crop['aux'].map(lambda x: True if ((x[0] == '') or (x[1] == '') or (x[2] == '') or (x[3] == '') or (x[4] == '')) else False) 
    elif conseq in ["frameshift", 'nonstop']: # (w, d, w, '', w) or (w, d, w, '', '')
        df_crop['badformat'] = df_crop['aux'].map(lambda x: True if ((x[0] == '') or (x[1] == '') or (x[2] == '') or (x[3] != '')) else False) 
        #bad format in nonstop for the delext* (w, d, '', '', w) or (w, d, '', '','')
    elif conseq in ['missense']: # (w, d, w, '', '') 
        df_crop['badformat'] = df_crop['aux'].map(lambda x: True if ((x[0] == '') or (x[1] == '') or (x[2] == '') or (x[3] != '') or (x[4] != '')) else False)  
    '''
    # start position
    df_crop['start_aa'] = df_crop['aux'].map(lambda x: x[1])
    # end position
    df_crop['end_aa'] = df_crop['aux'].map(lambda x: x[3] if x[3] != '' else x[1]) # poner en el end el start si no hay end
        
    # from: es el/los aa que cambian # concateno si existe mas de un aa que cambia (o sea, si es un rango)
    df_crop['from_aa'] = df_crop['aux'].map(lambda x: str(x[0]) + ('-' + str(x[2]) if x[2] != '' and x[3] != '' else '')) 
        
    if conseq == "missense":     
        df_crop['to_aa'] = df_crop['aux'].map(lambda x: x[2])
    elif conseq == "nonsense":
        df_crop['to_aa'] = "*"
    elif conseq in ["frameshift", 'nostop']:    
        df_crop['to_aa'] = ""
    else: # to: aa/s nuevos
        df_crop['to_aa'] = df_crop['aux'].map(lambda x: x[4])

    # consecuencia de la mutacion
    df_crop['consequence_protein'] = conseq
    #print(df_crop[df_crop['badformat'] == True].sort_values(by='AA', key=lambda col: col.str.len(), ascending=False, na_position='last'))
    
    df_crop = df_crop.drop(columns=['aux'])

    return df_crop

In [11]:
# A DataFrame for each molecular consequence
print("Mapping AA mutations")
mutation_used = mutation.copy()
print(f"all mutations {mutation.shape[0]}")

synonym = separar_en_cols(mutation_used, "AA", "synonym", "=$")
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(synonym['indexinternal'].tolist())]
print(f"Found {synonym.shape[0]} synonym")


delins = separar_en_cols(mutation_used, "AA", "delins", "delins")
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(delins['indexinternal'].tolist())]
print(f"Found {delins.shape[0]} delins")

deletions = separar_en_cols(mutation_used, "AA", "deletion", "del$") # finish with 'del'
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(deletions['indexinternal'].tolist())]
print(f"Found {deletions.shape[0]} deletions")

insertions = separar_en_cols(mutation_used, "AA", "insertion", "ins")
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(insertions['indexinternal'].tolist())]
print(f"Found {insertions.shape[0]} inserions")

frameshift = separar_en_cols(mutation_used, "AA", "frameshift", 'fs\*')
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(frameshift['indexinternal'].tolist())]
print(f"Found {frameshift.shape[0]} frameshift")

duplications = separar_en_cols(mutation_used, "AA", "duplication", "dup")
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(duplications['indexinternal'].tolist())]
print(f"Found {duplications.shape[0]} duplications")

nostop = separar_en_cols(mutation_used, "AA", "nonstop", '(?:del)?ext\*')
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(nostop['indexinternal'].tolist())]
print(f"Found {nostop.shape[0]} nostop")

nonsense = separar_en_cols(mutation_used, "AA", "nonsense", "\*$")
mutation_used = mutation_used[~mutation_used['indexinternal'].isin(nonsense['indexinternal'].tolist())]
print(f"Found {nonsense.shape[0]} nonsense")

missense = separar_en_cols(mutation_used, "AA", "missense", '^([A-Z])(\d+)([A-Z])$(.*)?(.*)?$', override=True)
print(f"Found {missense.shape[0]} missense")

#los frameshift M1?, M1_?\d*
## Concatenate everything
mutation_mapped = pd.concat([synonym, delins, deletions, insertions, frameshift, duplications, nostop, nonsense, missense])

print(f"mapped {mutation_mapped.shape[0]} mutations ")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(mutation_mapped['indexinternal'].tolist())]
print(f"No mapped {mutation_used.shape[0]} mutations")


Mapping AA mutations
all mutations 1144756
Found 264184 synonym
Found 1606 delins
Found 5403 deletions
Found 884 inserions
Found 38326 frameshift
Found 1031 duplications
Found 743 nostop
Found 60124 nonsense
Found 771186 missense
mapped 1143487 mutations 
No mapped 1269 mutations


In [12]:
def separar_en_cols_cds(df, column, conseq, conseq_regex, override=False):
    '''
    recibe un DataFrame, el nombre de una columna auxiliar (column)
    y un string con el tipo de consecuencia (conseq). La col. auxiliar
    es una tupla con los elementos implicados en una mutacion
    como la siguiente (nucleatides1, nucleatides2).
    Devuelve el DataFrame df con estas 2 nuevas columnas
    '''
    
    df_crop = df.copy()
      
      
    if override:
        df_crop['aux'] = df_crop[column].str.findall(conseq_regex).str[0]
    else:
        df_crop['aux'] = df_crop[column].str.findall('^[-\+\d]+_?(?:[-\+\d]+)?'+conseq_regex+'([ACTG]+)?>?([ACTG]+)?$').str[0]
  
    df_crop = df_crop[~df_crop['aux'].isnull()]
    
    df_crop['from_genomic'] = ''
    df_crop['to_genomic'] = ''
    
    if conseq in ["delins", 'insertion']: 
        df_crop['to_genomic'] = df_crop['aux'].map(lambda x: x[0])
    elif conseq == 'missense': 
        df_crop['from_genomic'] = df_crop['aux'].map(lambda x: x[0])
        df_crop['to_genomic'] = df_crop['aux'].map(lambda x: x[1])

    # consecuencia de la mutacion
    df_crop['consequence_genomic'] = conseq
    #print(df_crop[df_crop['badformat'] == True].sort_values(by='AA', key=lambda col: col.str.len(), ascending=False, na_position='last'))
    
    df_crop = df_crop.drop(columns=['aux'])

    return df_crop

In [14]:
print("Mapping CDS mutations")

mutation_used = mutation_mapped.copy()
print(f"all mutations {mutation_used.shape[0]}")

delins = separar_en_cols_cds(mutation_used, "CDS", "delins", "delins")
print(f"Found {delins.shape[0]} delins cds")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(delins['indexinternal'].tolist())]
deletions = separar_en_cols_cds(mutation_used, "CDS", "deletion", "del$") # finish with 'del'
print(f"Found {deletions.shape[0]} deletions cds")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(deletions['indexinternal'].tolist())]
insertions = separar_en_cols_cds(mutation_used, "CDS", "insertion", "ins")
print(f"Found {insertions.shape[0]} inserions cds")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(insertions['indexinternal'].tolist())]
duplications = separar_en_cols_cds(mutation_used, "CDS", "duplication", "dup$")
print(f"Found {duplications.shape[0]} duplications cds")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(duplications['indexinternal'].tolist())]
inversions = separar_en_cols_cds(mutation_used, "CDS", "invertion", "inv$")
print(f"Found {inversions.shape[0]} inversion cds")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(inversions['indexinternal'].tolist())]
substitution = separar_en_cols_cds(mutation_used, "CDS", "missense", '')
print(f"Found {missense.shape[0]} missense cds")

mutation_mapped2 = pd.concat([delins, deletions, insertions, duplications, inversions, substitution])

print(f"mapped {mutation_mapped2.shape[0]} mutations ")

mutation_used = mutation_used[~mutation_used['indexinternal'].isin(mutation_mapped2['indexinternal'].tolist())]
print(f"No mapped {mutation_used.shape[0]} mutations")

Mapping CDS mutations
all mutations 1143487
Found 9032 delins cds
Found 31123 deletions cds
Found 4077 inserions cds
Found 12871 duplications cds
Found 317 inversion cds
Found 771186 missense cds
mapped 1143487 mutations 
No mapped 0 mutations


In [30]:
print(mutation_mapped2.shape[0])
a = mutation_mapped2[['start_aa', 'end_aa', 'to_aa', 'from_aa', 'id_protein', 'consequence_protein', 'start_genomic', 'end_genomic', 'from_genomic', 'to_genomic', 'consequence_genomic']].drop_duplicates()
print(a.shape[0])



x = mutation_mapped2[['start_aa', 'end_aa', 'to_aa', 'from_aa', 'id_protein', 'consequence_protein']].drop_duplicates()
print(x.shape[0])

y = x.groupby(['start_aa', 'end_aa', 'to_aa', 'from_aa', 'id_protein']).size().reset_index(name='counts')
y = y[y.counts > 1]
y = y.sort_values(by='counts', ascending=False)
print(y)

x = mutation_mapped2[['start_aa', 'end_aa', 'to_aa', 'from_aa', 'id_protein']].drop_duplicates()
print(x.shape[0])

1143487
1143300
1119621
        start_aa end_aa to_aa from_aa  id_protein  counts
394797       241    241             S         623       4
69287       1157   1157             E        1020       4
413360       251    251             I         623       4
415060       252    252             L         623       4
416902       253    253             T         623       4
...          ...    ...   ...     ...         ...     ...
355178       221    221             P         242       2
355190       221    221             P        2201       2
356684      2216   2216             P         507       2
356685      2216   2216             Q        3004       2
1114290      997    997             Y        1020       2

[4392 rows x 6 columns]
1115104


In [31]:
print(len(y['id_protein'].unique()))
mutation_mapped2[((mutation_mapped2['id_protein'] == 623) & (mutation_mapped2['start_aa'] == '241') & (mutation_mapped2['end_aa'] == '241') & (mutation_mapped2['from_aa'] == 'S'))]

1789


,LEGACY_MUTATION_ID,id_protein,CDS,AA,consequence_cosmic,chromosome,start_genomic,end_genomic,indexinternal,start_aa,end_aa,from_aa,to_aa,consequence_protein,from_genomic,to_genomic,consequence_genomic
1496511,COSM45017,623,722_723delinsTT,S241F,Substitution - Missense,17,7674240,7674241,1039810,241,241,S,F,missense,,TT,delins
1497270,COSM45168,623,722_724del,S241del,Deletion - In frame,17,7674240,7674242,1039911,241,241,S,,deletion,,,deletion
1500937,COSM5966959,623,720_722del,S241del,Deletion - In frame,17,7674241,7674243,1040312,241,241,S,,deletion,,,deletion
1494914,COSM43645,623,721del,S241Pfs*6,Deletion - Frameshift,17,7674243,7674243,1039602,241,241,S,,frameshift,,,deletion
1499986,COSM9345425,623,722_732del,S241Wfs*19,Deletion - Frameshift,17,7674231,7674241,1040220,241,241,S,,frameshift,,,deletion
1519450,COSM45250,623,720_721del,S241Lfs*22,Deletion - Frameshift,17,7674242,7674243,1041742,241,241,S,,frameshift,,,deletion
1522620,COSM45788,623,722_723del,S241Lfs*22,Deletion - Frameshift,17,7674240,7674241,1041936,241,241,S,,frameshift,,,deletion
1526895,COSM9306816,623,721_722del,S241Lfs*22,Deletion - Frameshift,17,7674241,7674242,1042184,241,241,S,,frameshift,,,deletion
1494313,COSM6929598,623,722_724dup,S241dup,Insertion - In frame,17,7674242,7674243,1039516,241,241,S,,duplication,,,duplication
1500556,COSM46004,623,723C>T,S241=,Substitution - coding silent,17,7674240,7674240,1040282,241,241,S,,synonym,C,T,missense
